Requirements
1. Provide working source code that will :-
- a. For a given stock,
- i. Given any price as input, calculate the dividend yield
- ii. Given any price as input, calculate the P/E Ratio
- iii. Record a trade, with timestamp, quantity of shares, buy or sell indicator and
- traded price
- iv. Calculate Volume Weighted Stock Price based on trades in past 15 minutes
- b. Calculate the GBCE All Share Index using the geometric mean of prices for all stocks

Constraints & Notes
1. Written in one of these languages:
 Java, C#, C++, Python
2. No database or GUI is required, all data need only be held in memory
3. No prior knowledge of stock markets or trading is required – all formulas are provided below.

| Symbol | Type      | Last Dividend | Fixed Dividend | Par Value |
|--------|-----------|---------------|----------------|-----------|
| TEA    | Common    | 0             |                | 100       |
| POP    | Common    | 8             |                | 100       |
| ALE    | Common    | 23            |                | 60        |
| GIN    | Preferred | 8             | 2%             | 100       |
| JOE    | Common    | 13            |                | 250       |

$$
\text{Dividend Yield} = \frac{\text{Last Dividend}}{\text{Price}}
$$

$$
\text{P/E Ratio} = \frac{\text{Price}}{\text{Dividend}}
$$

$$
\text{Geometric Mean} = \sqrt{p_1 \times p_2 \times p_3 \times \ldots \times p_n}
$$

$$
\text{Volume Weighted Stock Price} = \frac{\sum_i \text{Traded Price}_i \times \text{Quantity}_i}{\sum_i \text{Quantity}_i}
$$

In [65]:
stock_data = [
    {
        "Symbol": "TEA",
        "Type": "Common",
        "Last Dividend": 0,
        "Fixed Dividend": None,
        "Par Value": 100
    },
    {
        "Symbol": "POP",
        "Type": "Common",
        "Last Dividend": 8.,
        "Fixed Dividend": None,
        "Par Value": 100
    },
    {
        "Symbol": "ALE",
        "Type": "Common",
        "Last Dividend": 23,
        "Fixed Dividend": None,
        "Par Value": 60
    },
    {
        "Symbol": "GIN",
        "Type": "Preferred",
        "Last Dividend": 8,
        "Fixed Dividend": .02,
        "Par Value": 100
    },
    {
        "Symbol": "JOE",
        "Type": "Common",
        "Last Dividend": 13,
        "Fixed Dividend": None,
        "Par Value": 250
    }
]


In [104]:
from scipy.stats import gmean

def calculate_gbce_all_share_index(stock_data):
    prices = [stock['Price'] for stock in stock_data if 'Price' in stock]
    if len(prices) == 0:
        return None
    gbce_all_share_index = gmean(prices)
    n = len(stock_data)
    gbce_all_share_index_root_n = gbce_all_share_index ** (1 / n)
    return gbce_all_share_index_root_n

In [117]:
from datetime import datetime, timedelta
from enum import Enum

class StockType(Enum):
    COMMON = "COMMON"
    PREFERRED = "PREFERRED"

class TradeType(Enum):
    BUY = "BUY"
    SELL = "SELL"

class SimpleData:
    def __init__(self, symbol: str, stock_type: StockType, last_dividend: float, fixed_dividend: float, par_value: float, trades=None):
        self.symbol = symbol
        self.stock_type = stock_type
        self.last_dividend = last_dividend
        self.fixed_dividend = fixed_dividend
        self.par_value = par_value
        self.trades = trades if trades else []

    def dividend_yield(self, price):
        if self.stock_type == StockType.COMMON:
            return self.last_dividend / price
        elif self.stock_type == StockType.PREFERRED:
            return (self.fixed_dividend * self.par_value) / price
        else:
            return None
    
    def pe_ratio(self, price):
        dividend = self.dividend_yield(price)
        if dividend is not None and dividend != 0:
            return price / dividend
        else:
            return None
    
    def record_trade(self, quantity: int, indicator: TradeType, price: float):
        timestamp = datetime.now()
        self.trades.append({'timestamp': timestamp, 'quantity': quantity, 'indicator': indicator, 'price': price})

    def volume_weighted_stock_price(self):
        # Get the current time
        current_time = datetime.now()

        # Calculate the start time for the past 15 minutes
        start_time = current_time - timedelta(minutes=15)

        # Filter trades within the past 15 minutes
        recent_trades = [trade for trade in self.trades if trade['timestamp'] >= start_time]

        # Calculate the total traded price and quantity
        total_traded_price = 0
        total_quantity = 0
        for trade in recent_trades:
            total_traded_price += trade['price'] * trade['quantity']
            total_quantity += trade['quantity']

        # Calculate the volume weighted stock price
        if total_quantity != 0:
            vwap = total_traded_price / total_quantity
            return vwap
        else:
            return None



In [118]:
# Example usage
#if __name__ == "__main__":
    # Creating an instance of SimpleData for common stock with initial trades
pop_trades = [
    {'timestamp': datetime.now(), 'quantity': 50, 'indicator': TradeType.BUY, 'price': 50.0},
    {'timestamp': datetime.now(), 'quantity': 30, 'indicator': TradeType.SELL, 'price': 60.0}
]
pop_stock = SimpleData("POP", StockType.COMMON, 8, 0, 100, trades=pop_trades)
price = 55.0
print(f"The dividend yield for POP stock with price ${price} is {pop_stock.dividend_yield(price)}")
print(f"The P/E ratio for POP stock with price ${price} is {pop_stock.pe_ratio(price)}")

# Record a new trade
pop_stock.record_trade(100, TradeType.BUY, 55.0)

# Update trades
new_trades = [
    {'timestamp': datetime.now(), 'quantity': 20, 'indicator': TradeType.SELL, 'price': 58.0},
    {'timestamp': datetime.now(), 'quantity': 70, 'indicator': TradeType.BUY, 'price': 62.0}
]
pop_stock.record_trade(120, TradeType.BUY, 58.0)

# Printing the updated trades list
print("Updated Trades List:")
for trade in pop_stock.trades:
    print(trade)

# Test dividend yield and P/E ratio
#price = 55.0
print(f"The dividend yield for POP stock with price ${price} is {pop_stock.dividend_yield(price)}")
print(f"The P/E ratio for POP stock with price ${price} is {pop_stock.pe_ratio(price)}")


The dividend yield for POP stock with price $55.0 is 0.14545454545454545
The P/E ratio for POP stock with price $55.0 is 378.125
Updated Trades List:
{'timestamp': datetime.datetime(2024, 2, 8, 20, 9, 21, 869251), 'quantity': 50, 'indicator': <TradeType.BUY: 'BUY'>, 'price': 50.0}
{'timestamp': datetime.datetime(2024, 2, 8, 20, 9, 21, 869251), 'quantity': 30, 'indicator': <TradeType.SELL: 'SELL'>, 'price': 60.0}
{'timestamp': datetime.datetime(2024, 2, 8, 20, 9, 21, 869251), 'quantity': 100, 'indicator': <TradeType.BUY: 'BUY'>, 'price': 55.0}
{'timestamp': datetime.datetime(2024, 2, 8, 20, 9, 21, 869251), 'quantity': 120, 'indicator': <TradeType.BUY: 'BUY'>, 'price': 58.0}
The dividend yield for POP stock with price $55.0 is 0.14545454545454545
The P/E ratio for POP stock with price $55.0 is 378.125


In [122]:
def calculate_gbce_all_share_index(stock_data):
    prices = [stock.volume_weighted_stock_price() for stock in stock_data]
    valid_prices = [price for price in prices if price is not None]
    if not valid_prices:
        return None
    return gmean(valid_prices)

In [123]:
stock_data = [
    SimpleData("TEA", StockType.COMMON, 0.0, 0.0, 100.0),
    SimpleData("POP", StockType.COMMON, 8.0, 0.0, 100.0),
    SimpleData("ALE", StockType.COMMON, 23.0, 0.0, 60.0),
    SimpleData("GIN", StockType.PREFERRED, 8.0, 0.2, 100.0),
    SimpleData("JOE", StockType.COMMON, 13.0, 0.0, 250.0)
]

# Test dividend yield and P/E ratio for a given price
test_price = 50.0
for stock in stock_data:
    print(f"Stock: {stock.symbol}")
    print(f"Dividend Yield for price {test_price}: {stock.dividend_yield(test_price)}")
    print(f"P/E Ratio for price {test_price}: {stock.pe_ratio(test_price)}")
    print()

# Record a trade for each stock
for stock in stock_data:
    stock.record_trade(10, 55.0, TradeType.BUY)
    stock.record_trade(5, 60.0, TradeType.SELL)

# Calculate Volume Weighted Stock Price for each stock
for stock in stock_data:
    print(f"Volume Weighted Stock Price for {stock.symbol}: {stock.volume_weighted_stock_price()}")

# Calculate GBCE All Share Index
gbce_all_share_index = calculate_gbce_all_share_index(stock_data)
print(f"\nGBCE All Share Index: {gbce_all_share_index}")

Stock: TEA
Dividend Yield for price 50.0: 0.0
P/E Ratio for price 50.0: None

Stock: POP
Dividend Yield for price 50.0: 0.16
P/E Ratio for price 50.0: 312.5

Stock: ALE
Dividend Yield for price 50.0: 0.46
P/E Ratio for price 50.0: 108.69565217391303

Stock: GIN
Dividend Yield for price 50.0: 0.4
P/E Ratio for price 50.0: 125.0

Stock: JOE
Dividend Yield for price 50.0: 0.26
P/E Ratio for price 50.0: 192.3076923076923



TypeError: unsupported operand type(s) for *: 'TradeType' and 'int'

In [119]:
gbce_all_share_index_root_n = calculate_gbce_all_share_index(stock_data)
if gbce_all_share_index_root_n is not None:
    print(f"GBCE All Share Index: {gbce_all_share_index_root_n}")
    # Perform assertion
    assert round(gbce_all_share_index_root_n, 10) == 1.3797296615  # Round to 10 decimal places for accurate comparison
else:
    print("No valid prices found in stock data.")

No valid prices found in stock data.


In [120]:
gbce_all_share_index_root_n()

In [190]:
from enum import Enum
from datetime import datetime, timedelta

from operator import mul
from functools import reduce

class StockType(Enum):
    COMMON = "COMMON"
    PREFERRED = "PREFERRED"

class TradeType(Enum):
    BUY = "BUY"
    SELL = "SELL"

class SimpleData:
    def __init__(self, symbol: str, stock_type: StockType, last_dividend: float, fixed_dividend: float, par_value: float):
        """
        Initialize a SimpleData object with the given parameters.

        :param symbol: The symbol of the stock.
        :param stock_type: The type of the stock (COMMON or PREFERRED).
        :param last_dividend: The last dividend paid by the stock.
        :param fixed_dividend: The fixed dividend rate for preferred stocks (percentage).
        :param par_value: The par value of the stock.
        """
        self.symbol = symbol
        self.stock_type = stock_type
        self.last_dividend = last_dividend
        self.fixed_dividend = fixed_dividend
        self.par_value = par_value
        self.trades = []
        
    def dividend_yield(self, price):
        """
        Calculate the dividend yield for the stock.

        Args:
            price (float): The current price of the stock.

        Returns:
            float: The calculated dividend yield.
            
        Raises:
            ValueError: If the price is non-positive.
        """
        # Check if price is non-positive
        if price <= 0:
            raise ValueError("Price must be positive")

        # Calculate dividend yield based on stock type
        if self.stock_type == StockType.COMMON:
            return self.last_dividend / price
        elif self.stock_type == StockType.PREFERRED:
            return (self.fixed_dividend * self.par_value) / price

    def pe_ratio(self, price):
        """
        Calculate the price-to-earnings (P/E) ratio.

        Args:
            price (float): The current price of the stock.

        Returns:
            float or None: The calculated P/E ratio, or None if the dividend is zero.
        """
        # Calculate dividend yield
        dividend = self.dividend_yield(price)
        
        # Return P/E ratio if dividend is non-zero, otherwise return None
        return price / dividend if dividend != 0 else None


    def record_trade(self, quantity: int, price: float, indicator: TradeType):
        """
        Record a trade for the stock.

        Args:
            quantity (int): The quantity of shares traded.
            price (float): The price at which the trade occurred.
            indicator (TradeType): Indicator whether the trade is a buy or sell.

        Returns:
            None
        """
        timestamp = datetime.now()
        self.trades.append({'timestamp': timestamp, 'quantity': quantity, 'price': price, 'indicator': indicator})

    def volume_weighted_stock_price(self):
        """
        Calculate the volume-weighted stock price based on trades within the last 15 minutes.

        Returns:
            float or None: The volume-weighted stock price, or None if there are no trades within the last 15 minutes.
        """
        # Calculate the timestamp 15 minutes ago
        fifteen_minutes_ago = datetime.now() - timedelta(minutes=15)

        # Filter trades within the last 15 minutes
        trades_within_15_minutes = [trade for trade in self.trades if trade['timestamp'] >= fifteen_minutes_ago]

        # If no trades within the last 15 minutes, return None
        if not trades_within_15_minutes:
            return None

        # Calculate total value and total quantity for volume-weighted stock price
        total_value = sum(trade['price'] * trade['quantity'] for trade in trades_within_15_minutes)
        total_quantity = sum(trade['quantity'] for trade in trades_within_15_minutes)

        # Calculate volume-weighted stock price
        return total_value / total_quantity


class GBCECalculator:
    @staticmethod
    def calculate_gbce_all_share_index(stock_data):
        """
        Calculate the GBCE All Share Index using the geometric mean of prices for all valid stocks.

        Args:
            stock_data (list): A list of stock data, each element containing information about a stock.

        Returns:
            float or None: The calculated GBCE All Share Index, or None if there are no valid stocks with trades.
        """
        if not stock_data:
            return None  # Handle case when stock_data is empty

        # Filter out stocks without any trades
        valid_stocks = [stock for stock in stock_data if stock.trades]

        if not valid_stocks:
            return None  # Handle case when all stocks have no trades

        # Calculate the product of prices for valid stocks
        prices = [stock.trades[-1]['price'] for stock in valid_stocks]
        product = reduce(mul, prices, 1)

        # Calculate the geometric mean of prices for all valid stocks
        return (product) ** (1.0 / len(valid_stocks))


# Example usage
if __name__ == "__main__":
    # Sample stock data
    stock_data = [
        SimpleData("TEA", StockType.COMMON, 0.0, 0.0, 100.0),
        SimpleData("POP", StockType.COMMON, 8.0, 0.0, 100.0),
        SimpleData("ALE", StockType.COMMON, 23.0, 0.0, 60.0),
        SimpleData("GIN", StockType.PREFERRED, 8.0, 0.2, 100.0),
        SimpleData("JOE", StockType.COMMON, 13.0, 0.0, 250.0)
    ]

    # Test dividend yield and P/E ratio for a given price
    test_price = 50.0
    for stock in stock_data:
        print(f"Stock: {stock.symbol}")
        print(f"Dividend Yield for price {test_price}: {stock.dividend_yield(test_price)}")
        print(f"P/E Ratio for price {test_price}: {stock.pe_ratio(test_price)}")
    # Record a trade for each stock
    for stock in stock_data:
        stock.record_trade(10, 55.0, TradeType.BUY)
        stock.record_trade(5, 60.0, TradeType.SELL)

    # Calculate Volume Weighted Stock Price for each stock
    for stock in stock_data:
        print(f"Volume Weighted Stock Price for {stock.symbol}: {stock.volume_weighted_stock_price()}")

    # Calculate GBCE All Share Index
    gbce_all_share_index = GBCECalculator.calculate_gbce_all_share_index(stock_data)
    print(f"\nGBCE All Share Index: {gbce_all_share_index}")

Stock: TEA
Dividend Yield for price 50.0: 0.0
P/E Ratio for price 50.0: None
Stock: POP
Dividend Yield for price 50.0: 0.16
P/E Ratio for price 50.0: 312.5
Stock: ALE
Dividend Yield for price 50.0: 0.46
P/E Ratio for price 50.0: 108.69565217391303
Stock: GIN
Dividend Yield for price 50.0: 0.4
P/E Ratio for price 50.0: 125.0
Stock: JOE
Dividend Yield for price 50.0: 0.26
P/E Ratio for price 50.0: 192.3076923076923
Volume Weighted Stock Price for TEA: 56.666666666666664
Volume Weighted Stock Price for POP: 56.666666666666664
Volume Weighted Stock Price for ALE: 56.666666666666664
Volume Weighted Stock Price for GIN: 56.666666666666664
Volume Weighted Stock Price for JOE: 56.666666666666664

GBCE All Share Index: 60.000000000000014


In [191]:
stock_data = [
    SimpleData("TEA", StockType.COMMON, 0.0, 0.0, 100.0),
    SimpleData("POP", StockType.COMMON, 8.0, 0.0, 100.0),
    SimpleData("ALE", StockType.COMMON, 23.0, 0.0, 60.0),
    SimpleData("GIN", StockType.PREFERRED, 8.0, 0.2, 100.0),
    SimpleData("JOE", StockType.COMMON, 13.0, 0.0, 250.0)
]

# Record trades for each stock
for stock in stock_data:
    # Buy and sell 10 times for each stock
    for _ in range(10):
        stock.record_trade(quantity=1, price=1.0, indicator=TradeType.BUY)
        stock.record_trade(quantity=1, price=1.0, indicator=TradeType.SELL)

# Calculate metrics for all stocks
for stock in stock_data:
    print(f"Stock: {stock.symbol}")
    print(f"Dividend Yield: {stock.dividend_yield(1.0)}")
    print(f"P/E Ratio: {stock.pe_ratio(1.0)}")
    print(f"Volume Weighted Stock Price: {stock.volume_weighted_stock_price()}")
    print(calculate_gbce_all_share_index(stock_data))

Stock: TEA
Dividend Yield: 0.0
P/E Ratio: None
Volume Weighted Stock Price: 1.0
1.0
Stock: POP
Dividend Yield: 8.0
P/E Ratio: 0.125
Volume Weighted Stock Price: 1.0
1.0
Stock: ALE
Dividend Yield: 23.0
P/E Ratio: 0.043478260869565216
Volume Weighted Stock Price: 1.0
1.0
Stock: GIN
Dividend Yield: 20.0
P/E Ratio: 0.05
Volume Weighted Stock Price: 1.0
1.0
Stock: JOE
Dividend Yield: 13.0
P/E Ratio: 0.07692307692307693
Volume Weighted Stock Price: 1.0
1.0


In [158]:
len(stock.trades)

20

In [159]:
prices = [stock.trades[-1]['price'] for stock in stock_data]

In [160]:
prices

[1.0, 1.0, 1.0, 1.0, 1.0]

In [192]:
# Define function to test scenarios
def test_scenarios():
    # Define scenarios
    scenarios = [
        # Scenario 1: Stock with Common Type
        {'symbol': 'A', 'stock_type': StockType.COMMON, 'last_dividend': 0.50, 'fixed_dividend': None, 'par_value': None, 'price': 10.0},

        # Scenario 2: Stock with Preferred Type
        # {'symbol': 'B', 'stock_type': StockType.PREFERRED, 'last_dividend': 2.00, 'fixed_dividend': 0.05, 'par_value': 100.0, 'price': 50.0},

        # Scenario 3: Stock with No Dividend (Common Type)
        # {'symbol': 'C', 'stock_type': StockType.COMMON, 'last_dividend': 0, 'fixed_dividend': None, 'par_value': None, 'price': 20.0},

        # Scenario 4: Stock with No Dividend (Preferred Type)
        # {'symbol': 'D', 'stock_type': StockType.PREFERRED, 'last_dividend': 0, 'fixed_dividend': 0.05, 'par_value': 100.0, 'price': 30.0},

        # Scenario 5: Stock with Very High Dividend (Common Type)
        # {'symbol': 'E', 'stock_type': StockType.COMMON, 'last_dividend': 100.0, 'fixed_dividend': None, 'par_value': None, 'price': 10.0}
    ]

    # Create SimpleData objects for each scenario and test dividend yield and P/E ratio
    for scenario in scenarios:
        stock = SimpleData(scenario['symbol'], scenario['stock_type'], scenario['last_dividend'], scenario['fixed_dividend'], scenario['par_value'])
        price = scenario['price']
        dividend_yield = stock.dividend_yield(price)
        pe_ratio = stock.pe_ratio(price)
        print(f"Scenario: {scenario['symbol']}")
        print(f"Price: {price}")
        print(f"Dividend Yield: {dividend_yield}")
        print(f"P/E Ratio: {pe_ratio}")
        print()

# Run the test scenarios
test_scenarios()

Scenario: A
Price: 10.0
Dividend Yield: 0.05
P/E Ratio: 200.0



In [183]:
scenarios = [
    # Scenario 1: Stock with Common Type
    {'symbol': 'A', 'stock_type': StockType.COMMON, 'last_dividend': 0.50, 'fixed_dividend': None, 'par_value': None, 'price': 10.0},

    # Scenario 2: Stock with Preferred Type
    {'symbol': 'B', 'stock_type': StockType.PREFERRED, 'last_dividend': 2.00, 'fixed_dividend': 0.05, 'par_value': 100.0, 'price': 50.0},

    # Scenario 3: Stock with No Dividend (Common Type)
    {'symbol': 'C', 'stock_type': StockType.COMMON, 'last_dividend': 0, 'fixed_dividend': None, 'par_value': None, 'price': 20.0},

    # Scenario 4: Stock with No Dividend (Preferred Type)
    {'symbol': 'D', 'stock_type': StockType.PREFERRED, 'last_dividend': 0, 'fixed_dividend': 0.05, 'par_value': 100.0, 'price': 30.0},

    # Scenario 5: Stock with Very High Dividend (Common Type)
    {'symbol': 'E', 'stock_type': StockType.COMMON, 'last_dividend': 100.0, 'fixed_dividend': None, 'par_value': None, 'price': 10.0}
]

In [184]:
scenario = scenarios[1]

In [185]:
stock = SimpleData(scenario['symbol'], scenario['stock_type'], scenario['last_dividend'], scenario['fixed_dividend'], scenario['par_value'])

In [178]:
stock.last_dividend

0.5

In [186]:
price = scenario['price']
dividend_yield = stock.dividend_yield(price)

In [187]:
price = scenario['price']
dividend_yield = stock.dividend_yield(price)
pe_ratio = stock.pe_ratio(price)
print(f"Scenario: {scenario['symbol']}")
print(f"Price: {price}")
print(f"Dividend Yield: {dividend_yield}")
print(f"P/E Ratio: {pe_ratio}")
print()

Scenario: B
Price: 50.0
Dividend Yield: 0.1
P/E Ratio: 500.0



In [189]:
stock.record_trade(10, 100, 'BUY')
stock.record_trade(5, 110, 'SELL')
#self.preferred_stock.record_trade(5, 120, 'BUY')
#self.preferred_stock.record_trade(10, 125, 'SELL')

500.0

In [180]:
dividend_yield

0.05

In [181]:
stock.pe_ratio(price)

200.0

In [ ]:
dividend = self.dividend_yield(price)
        return price / dividend